In [1]:
# Imports

from astropy import units as u
import numpy as np
from math import pi

from poliastro.bodies import Earth, Mars, Sun
from poliastro.twobody import Orbit

# Needed to handle TLE into Poliastro's Orbit
from tletools import TLE

# Needed for defining manouvers
from poliastro.maneuver import Maneuver
from poliastro.twobody import thrust
from poliastro.twobody.propagation import cowell

from perylune.orbit_tools import *

In [9]:
# Create circular, equatorial orbit:

orb1 = Orbit.circular(Earth, 300 * u.km, 0*u.deg)
orb2 = Orbit.circular(Earth, 300 * u.km, 60*u.deg)

# Calculate the delta-v cost of inclination change from 87 to 88.4 degrees
dv_inc = inc_change(orb1, orb2)
print("Inclination change from %s to %s requires delta-v %s" % (orb1.inc, orb2.inc, dv_inc))

Inclination change from 0.0 deg to 60.0 deg requires delta-v 7725.760463451861 m / s


In [11]:
# Now calculate the cost for 60 deg change for eccentric orbits (300km)
alt = 300*u.km
orb3 = Orbit.from_classical(Earth, Earth.R + alt, 0.1*u.one, 0.0*u.deg, 0*u.deg, 0*u.deg, 0*u.deg)
orb4 = Orbit.from_classical(Earth, Earth.R + alt, 0.1*u.one, 60.0*u.deg, 0*u.deg, 0*u.deg, 0*u.deg)
print_orb(orb3)

6010 x 7346 km x 0.0 deg (GCRS) orbit around Earth (♁) at epoch J2000.000 (TT)
𝑎=6678.14km, b=6644.66km, e=0.10, i=0.00deg raan(Ω)=0.00deg argp(𝜔)=0.00deg nu(𝜈)=0.00deg period=5431.176641180842 s perapis=6010.32294 km apoapsis=7345.95026 km


In [24]:
# Calculate velocity vector magnitudes for 100 intervals, each 5 seconds
samples = 50
delta_t = orb3.period / samples
vel = calc_vel_inc_cost(orb3, orb4, delta_t, samples)
for v,t,nu,argp, dv in vel:
    nu2pi = nu + 2*pi*u.rad*(nu<0) # nu normalized to 0..2pi
    time_to_periapis = argp - nu2pi
    print("v=%4.2f %s at %s, nu=%s time to argp=%s, delta-v cost=%s" % (v.value, v.unit, t.iso, nu, (time_to_periapis), abs(dv)))

v=8.54 km / s at 2000-01-01 12:01:00.000, nu=0.0852460276884015 rad time to argp=-0.08524602768840145 rad, delta-v cost=6.965136658338098 km / s
v=8.52 km / s at 2000-01-01 12:02:48.624, nu=0.2392129425144507 rad time to argp=-0.2392129425144507 rad, delta-v cost=6.806842812334337 km / s
v=8.49 km / s at 2000-01-01 12:04:37.247, nu=0.39216523891136656 rad time to argp=-0.39216523891136573 rad, delta-v cost=6.502571317224405 km / s
v=8.44 km / s at 2000-01-01 12:06:25.871, nu=0.5435017853067654 rad time to argp=5.73968352187282 rad, delta-v cost=6.060625456173863 km / s
v=8.38 km / s at 2000-01-01 12:08:14.494, nu=0.6926922623334821 rad time to argp=5.590493044846103 rad, delta-v cost=5.492718946067656 km / s
v=8.30 km / s at 2000-01-01 12:10:03.118, nu=0.8392945823397118 rad time to argp=5.443890724839873 rad, delta-v cost=4.8133223134493095 km / s
v=8.22 km / s at 2000-01-01 12:11:51.741, nu=0.9829645096257495 rad time to argp=5.300220797553834 rad, delta-v cost=4.038925321744704 km /

In [25]:
# This is how the orbit can be propagated (the object moving forward in time)
# Observing how the velocity and position vectors change over time
print("orb1 v=%s, r=%s" % (orb1.v, orb1.r))
orb1 = orb1.propagate(1 * u.min) # Let's see how far it moved in 1 minute
print("orb3 v=%s, r=%s" % (orb1.v, orb1.r))

orb1 v=[-0.53583321  7.70715625  0.        ] km / s, r=[6662.05514628  463.17348383    0.        ] km
orb3 v=[-1.06908577  7.65143322  0.        ] km / s, r=[6613.88823583  924.11625469    0.        ] km


In [43]:
import plotly.express as px
import numpy as np

# y - blue - orbital velocity [km/s]
# y2 - red - delta-v cost for inclination change [km/s]

# Note the lowest cost is when the object passes lines of apides (i.e. in the middle between apoapsis and periapsis)

x = []
y = []
y2 = []
for v,t,nu,argp, dv in vel:
    x.append(nu.value)
    y.append(v.value)
    y2.append(abs(dv.value))

df = px.data.iris()
fig = px.scatter(x=x, y=[y,y2], labels={'x':'mean anomaly [0..2pi]', 'wide_variable_0': 'velocity[km/s]'})

fig.show()
